In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("index.txt"), sep="\t", header=True, inferSchema=True)
df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------+
|SAMPLE CONTENT:                                                                                          |
+---------------------------------------------------------------------------------------------------------+
|https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv                                            |
|https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv                                            |
|DATA COLUMNS:                                                                                            |
|marketplace       - 2 letter country code of the marketplace where the review was written.               |
|customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  |
|review_id         - The unique ID of the review.                                                         |
|product_id        - The uni

In [17]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("sample_us.tsv"), sep="\t", header=True, inferSchema=True)
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|cus

In [18]:
from pyspark.sql.functions import col, avg
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [19]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
gt20_df = vine_df.filter(vine_df['total_votes'] >= 20)
gt20_df.show(truncate=False)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [20]:
# 1.1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 1
gt1_df = vine_df.filter(vine_df['total_votes'] >= 1)
gt1_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UE3RPRGCOLD |2          |1            |1          |N   |Y                |
|RNX4EXOBBPN5  |1          |1            |1          |N   |Y                |
|R3SORMPJZO3F2J|3          |2            |2          |N   |Y                |
|R1CB783I7B0U52|1          |0            |1          |N   |Y                |
|R2BUV9QJI2A00X|5          |0            |1          |N   |Y                |
|R1T96CG98BBA15|3          |2            |4          |N   |Y                |
|R1YS3DS218NNMD|5          |4            |4          |N   |N                |
|R4R337CCDWLNG |3          |0            |1          |Y   |N                |
|R1H1HOVB44808I|5          |1            |1          |N   |N                |
|R4UVQIRZ5T1FM |4          |1            |2          |N   |Y    

In [22]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows 
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
pc50_df = gt1_df.filter(gt1_df["helpful_votes"]/gt1_df["total_votes"] >= 0.5)
pc50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R1T96CG98BBA15|          3|            2|          4|   N|                Y|
|R1YS3DS218NNMD|          5|            4|          4|   N|                N|
|R1H1HOVB44808I|          5|            1|          1|   N|                N|
| R4UVQIRZ5T1FM|          4|            1|          2|   N|                Y|
|R3031Q42BKAN7J|          4|            1|          1|   N|                N|
| R44NP0QG6E98W|          3|            1|          1|   N|                Y|
| RKLAK7EPEG5S6|          5|            1|          2|   N|     

In [26]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_review_yes_df = pc50_df.filter(pc50_df['vine']== 'Y')
vine_review_yes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [27]:
#   4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_review_no_df = pc50_df.filter(pc50_df['vine']== 'N')
vine_review_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R1T96CG98BBA15|          3|            2|          4|   N|                Y|
|R1YS3DS218NNMD|          5|            4|          4|   N|                N|
|R1H1HOVB44808I|          5|            1|          1|   N|                N|
| R4UVQIRZ5T1FM|          4|            1|          2|   N|                Y|
|R3031Q42BKAN7J|          4|            1|          1|   N|                N|
| R44NP0QG6E98W|          3|            1|          1|   N|                Y|
| RKLAK7EPEG5S6|          5|            1|          2|   N|     

In [29]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
vine_number = vine_review_yes_df.count() + 1
vine_five_star_number = vine_review_yes_df.filter(vine_review_yes_df['star_rating']== 5).count()
percentage_five_star_vine = float(vine_five_star_number) / float(vine_number)
print(vine_number)
print(vine_five_star_number)
print(percentage_five_star_vine)

1
0
0.0


In [32]:
no_vine_number = vine_review_no_df.count()
no_vine_five_star_number = vine_review_no_df.filter(vine_review_no_df['star_rating']== 5).count()
pct_no_five_star_vine = float(no_vine_five_star_number) / float(no_vine_number)
print(no_vine_number)
print(no_vine_five_star_number)
print(pct_no_five_star_vine)

12
4
0.3333333333333333
